In [2]:
print("Hello")

Hello


In [ ]:
import pyspark

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf

AttributeError: module 'socketserver' has no attribute 'UnixStreamServer'

In [2]:
spark = (
    SparkSession
    .builder
    .appName("PySpark tutorial")
    .master("local[*]")
    .getOrCreate()
)

NameError: name 'SparkSession' is not defined

#### Read/Write

In [50]:
data = [
    (1, "Alice", 20),
    (2, "Bob", 25),
    (3, "Charlie", 30),
    (4, "David", 35)
]

columns = ["id", "name", "age"]

df = spark.createDataFrame(data, columns)

df.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 20|
|  2|    Bob| 25|
|  3|Charlie| 30|
|  4|  David| 35|
+---+-------+---+



In [ ]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [ ]:
schema = st.StructType(
    [
        st.StructField("id", st.IntegerType(), True),
        st.StructField("name", st.StringType(), True),
        st.StructField("age", st.IntegerType(), True),
    ]
)

df = spark.createDataFrame(data, schema=schema)

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [ ]:
df_read_csv = spark.read.csv("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\2010-summary.csv", header=True)

df_read_csv.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [ ]:
df_read_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("delimiter", ",")
    .load("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\2010-summary.csv")
)

df_read_csv.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [ ]:
df_read_parquet = (
    spark
    .read
    .format("parquet")
    .load("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\yellow_tripdata_2024-09.parquet")
)

df_read_parquet.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|                 N|         138|          48|           1|       47.8|10.25|    0.5|      13.

In [52]:
df_read_parquet.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [53]:
df_read_parquet.count()

3633030

In [54]:
df_read_parquet.write.parquet("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\yellow_tripdata.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/C:/Users/SZABGAB/cubix_data_engineer_pyspark_tutorial/src/data/yellow_tripdata.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

#### Transformation

In [81]:
taxi_df = (
    spark
    .read
    .format("parquet")
    .load("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\yellow_tripdata_2024-09.parquet")
)

In [42]:
taxi_df.describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+------------------+---------------------+------------------+--------------------+-------------------+
|summary|          VendorID|   passenger_count|    trip_distance|        RatecodeID|store_and_fwd_flag|     PULocationID|     DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|       tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee|
+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+------------------+---------------------+------------------+--------------------+-------------

In [45]:
taxi_df.select("VendorID", "passenger_count").show(5)

+--------+---------------+
|VendorID|passenger_count|
+--------+---------------+
|       1|              1|
|       1|              1|
|       2|              2|
|       2|              1|
|       2|              2|
+--------+---------------+
only showing top 5 rows



In [47]:
taxi_df.select(taxi_df.VendorID, taxi_df.passenger_count).show(5)

+--------+---------------+
|VendorID|passenger_count|
+--------+---------------+
|       1|              1|
|       1|              1|
|       2|              2|
|       2|              1|
|       2|              2|
+--------+---------------+
only showing top 5 rows



In [67]:
taxi_df.select(sf.col("VendorID"), sf.col("passenger_count")).show(5)

+--------+---------------+
|VendorID|passenger_count|
+--------+---------------+
|       1|              1|
|       1|              1|
|       2|              2|
|       2|              1|
|       2|              2|
+--------+---------------+
only showing top 5 rows



In [ ]:
#### Selecting

In [ ]:
(
    taxi_df
    .where(
        (sf.col("trip_distance") > 5)
        & (sf.col("passenger_count") > 1)
    )
    .show(5)
)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:08:28|  2024-09-01 00:39:06|              4|          9.8|         1|                 N|          93|         161|           1|       44.3|  3.5|    0.5|      9.8

In [61]:
taxi_df.where(sf.col("store_and_fwd_flag") == 'N').show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|                 N|         138|          48|           1|       47.8|10.25|    0.5|      13.

In [62]:
taxi_df.where(sf.col("store_and_fwd_flag") != 'N').show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:23:40|  2024-09-01 00:43:46|              1|          3.9|         1|                 Y|         142|          42|           1|       20.5|  3.5|    0.5|      6.3

In [78]:
(
    taxi_df
    .where(
        ~(sf.col("store_and_fwd_flag") != 'N')
    )
    .show(5)
)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|total_amount_with_all_tax|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|               

#### withColumn

In [91]:
taxi_df = (
    taxi_df
    .withColumn(
        "total_amount_with_all_tax",
        sf.col("total_amount") + sf.col("congestion_surcharge") + sf.col("Airport_fee")
    )
)

(
    taxi_df
    .select(
        sf.col("total_amount"),
        sf.col("congestion_surcharge"),
        sf.col("Airport_fee"),        
        sf.col("total_amount_with_all_tax")
    )
).show(5)


+------------+--------------------+-----------+-------------------------+
|total_amount|congestion_surcharge|Airport_fee|total_amount_with_all_tax|
+------------+--------------------+-----------+-------------------------+
|       79.79|                 2.5|       1.75|                    84.04|
|        13.1|                 2.5|        0.0|                     15.6|
|        16.0|                 0.0|        0.0|                     16.0|
|       31.75|                 0.0|        0.0|                    31.75|
|        26.4|                 2.5|        0.0|                     28.9|
+------------+--------------------+-----------+-------------------------+
only showing top 5 rows



#### withColumnRenamed

In [86]:
taxi_df = taxi_df.withColumnRenamed("total_amount_with_all_tax", "total_amount_with_taxes")
taxi_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|total_amount_with_taxes|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------------------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|                 N|  

#### Drop

In [93]:
taxi_df = taxi_df.drop(sf.col("total_amount_with_all_tax"))
taxi_df.show(2)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|                 N|         138|          48|           1|       47.8|10.25|    0.5|      13.

#### Group by (order by)

In [95]:
(
    taxi_df
    .groupBy(sf.col("PULocationID"))
    .count()
    .sort(sf.col("count"), ascending=False)
    .show(10)
)

+------------+------+
|PULocationID| count|
+------------+------+
|         132|188147|
|         237|166033|
|         161|153988|
|         236|149284|
|         186|120393|
|         162|117163|
|         230|113531|
|         138|111112|
|         142|107537|
|          68|104585|
+------------+------+
only showing top 10 rows



In [ ]:
(
    taxi_df
    .groupBy(sf.col("payment_type"))
    .agg(
        sf.sum("total_amount").alias("total_amount"),
        sf.avg("total_amount").alias("average_amount")
    )
    .sort(sf.col("total_amount"), ascending=False)
    .withColumn("total_amount", sf.format_number(sf.col("total_amount"), 2)) # converted to string with the transformation
    .withColumn("average_amount", sf.format_number(sf.col("average_amount"), 2)) # converted to string with the transformation
    .show()
)

+------------+-------------+--------------+
|payment_type| total_amount|average_amount|
+------------+-------------+--------------+
|           1|81,090,276.01|         31.13|
|           0|11,577,187.59|         23.93|
|           2|10,682,143.65|         23.99|
|           3|   198,527.68|          7.84|
|           4|   135,263.93|          1.83|
+------------+-------------+--------------+



: 

#### Joins

In [3]:
# Dataframe 1: employees
employees_data = [
    (1, "Alice", "101"),
    (2, "Bob", "102"),
    (3, "Charlie", "103"),
    (4, "David", "101"),
    (5, "Eve", "104"),
    (6, "Frank", "105"),
    (7, "Grace", "102"),
    (8, "Helen", "106"),
    (9, "Ian", "103"),
    (10, "Jack", "104")
]
employees_columns = ["employee_id", "name", "department_id"]
employees_df = spark.createDataFrame(employees_data, employees_columns)

# Dataframe 2: departments
departments_data = [
    (101, "HR"),
    (102, "Finance"),
    (103, "IT"),
    (104, "Marketing"),
    (105, "Sales"),
    (107, "Operations")
]
departments_columns = ["department_id", "department_name"]
departments_df = spark.createDataFrame(departments_data, departments_columns)

print(employees_df.show())
print(departments_df.show())

+-----------+-------+-------------+
|employee_id|   name|department_id|
+-----------+-------+-------------+
|          1|  Alice|          101|
|          2|    Bob|          102|
|          3|Charlie|          103|
|          4|  David|          101|
|          5|    Eve|          104|
|          6|  Frank|          105|
|          7|  Grace|          102|
|          8|  Helen|          106|
|          9|    Ian|          103|
|         10|   Jack|          104|
+-----------+-------+-------------+

None
+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|          101|             HR|
|          102|        Finance|
|          103|             IT|
|          104|      Marketing|
|          105|          Sales|
|          107|     Operations|
+-------------+---------------+

None


In [4]:
inner_join_df = (
    employees_df
    .join(departments_df, "department_id", how="inner")
    .sort(sf.col("department_id"))
)

inner_join_df.show()

+-------------+-----------+-------+---------------+
|department_id|employee_id|   name|department_name|
+-------------+-----------+-------+---------------+
|          101|          1|  Alice|             HR|
|          101|          4|  David|             HR|
|          102|          2|    Bob|        Finance|
|          102|          7|  Grace|        Finance|
|          103|          3|Charlie|             IT|
|          103|          9|    Ian|             IT|
|          104|          5|    Eve|      Marketing|
|          104|         10|   Jack|      Marketing|
|          105|          6|  Frank|          Sales|
+-------------+-----------+-------+---------------+



In [6]:
left_join_df = (
    employees_df
    .join(departments_df, "department_id", how="left")
    .sort(sf.col("department_id"))
)

left_join_df.show()

+-------------+-----------+-------+---------------+
|department_id|employee_id|   name|department_name|
+-------------+-----------+-------+---------------+
|          101|          4|  David|             HR|
|          101|          1|  Alice|             HR|
|          102|          7|  Grace|        Finance|
|          102|          2|    Bob|        Finance|
|          103|          9|    Ian|             IT|
|          103|          3|Charlie|             IT|
|          104|         10|   Jack|      Marketing|
|          104|          5|    Eve|      Marketing|
|          105|          6|  Frank|          Sales|
|          107|       NULL|   NULL|     Operations|
+-------------+-----------+-------+---------------+



In [7]:
outer_join_df = (
    employees_df
    .join(departments_df, "department_id", how="outer")
    .sort(sf.col("department_id"))
)

outer_join_df.show()

+-------------+-----------+-------+---------------+
|department_id|employee_id|   name|department_name|
+-------------+-----------+-------+---------------+
|          101|          1|  Alice|             HR|
|          101|          4|  David|             HR|
|          102|          2|    Bob|        Finance|
|          102|          7|  Grace|        Finance|
|          103|          3|Charlie|             IT|
|          103|          9|    Ian|             IT|
|          104|          5|    Eve|      Marketing|
|          104|         10|   Jack|      Marketing|
|          105|          6|  Frank|          Sales|
|          106|          8|  Helen|           NULL|
|          107|       NULL|   NULL|     Operations|
+-------------+-----------+-------+---------------+



#### SparkSQL

In [9]:
taxi_df = (
    spark
    .read
    .format("parquet")
    .load("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\yellow_tripdata_2024-09.parquet")
)

In [12]:
taxi_df.createOrReplaceTempView("taxi_temp_view")

In [13]:
spark.sql("SELECT * FROM taxi_temp_view LIMIT 10").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:05:51|  2024-09-01 00:45:03|              1|          9.8|         1|                 N|         138|          48|           1|       47.8|10.25|    0.5|      13.

In [15]:
spark.sql("""
    SELECT
        *
    FROM
        taxi_temp_view
    WHERE
        trip_distance > 10
""").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-09-01 00:57:30|  2024-09-01 01:40:20|              2|         14.5|         1|                 N|         132|          91|           1|       64.6| 2.75|    0.5|     13.7

NameError: name 'avg_amount_by_payment_type' is not defined

In [17]:
avg_amount_by_payment_type = spark.sql("""
    SELECT
        payment_type,
        avg(total_amount)
    FROM
        taxi_temp_view
    GROUP BY
        payment_type
""")

avg_amount_by_payment_type.show()



+------------+------------------+
|payment_type| avg(total_amount)|
+------------+------------------+
|           1|31.132636716912728|
|           3| 7.835484864032835|
|           2|23.992416616129827|
|           4|1.8263854120252212|
|           0|23.933110737169443|
+------------+------------------+



In [18]:
# createOrReplaceTempView() or any other view creation only supports SELECTing, no UPDATE/INSERT/DELETE

spark.sql("""
    UPDATE taxi_temp_view
    SET passenger_count = 100
    WHERE
        VendorID = 1
        AND trip_distance = 14.5
""")

UnsupportedOperationException: UPDATE TABLE is not supported temporarily.

#### Caching

In [14]:
import time

In [15]:
data = [(i, i * 2) for i in range(100000)]
df = spark.createDataFrame(data, ["number", "double"])
df.show(5)

start_time = time.time()

# Repeat an expensive computation twice
df_filtered = df.filter(df["number"] % 2 == 0)
print(df_filtered.count()) # First computation
print(df_filtered.count()) # Second computation

end_time = time.time()
print(f"Time taken without caching: {end_time - start_time:.2f} seconds")

+------+------+
|number|double|
+------+------+
|     0|     0|
|     1|     2|
|     2|     4|
|     3|     6|
|     4|     8|
+------+------+
only showing top 5 rows

50000
50000
Time taken without caching: 27.64 seconds


#### Broadcast

In [16]:
product_ids = [i for i in range(1, 3000001)]
product_names = [f"Product_{i}" for i in range (1, 3000001)]

large_data = list(zip(product_ids, product_names))
columns = ["product_id", "product_names"]

large_df = spark.createDataFrame(large_data, columns)
large_df.show(5)

category_data = [(1, "Electronics"), (2, "Clothing"), (3, "Toys"), (4, "Groceries"), (5, "Books")]
category_columns = ["product_id", "category"]

small_df = spark.createDataFrame(category_data, category_columns)
small_df.show(5)


+----------+-------------+
|product_id|product_names|
+----------+-------------+
|         1|    Product_1|
|         2|    Product_2|
|         3|    Product_3|
|         4|    Product_4|
|         5|    Product_5|
+----------+-------------+
only showing top 5 rows

+----------+-----------+
|product_id|   category|
+----------+-----------+
|         1|Electronics|
|         2|   Clothing|
|         3|       Toys|
|         4|  Groceries|
|         5|      Books|
+----------+-----------+



In [17]:
start_time = time.time()

joined_df_no_broadcast = large_df.join(small_df, "product_id")
joined_df_no_broadcast.show(5)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

+----------+-------------+-----------+
|product_id|product_names|   category|
+----------+-------------+-----------+
|         1|    Product_1|Electronics|
|         5|    Product_5|      Books|
|         3|    Product_3|       Toys|
|         2|    Product_2|   Clothing|
|         4|    Product_4|  Groceries|
+----------+-------------+-----------+

Time taken: 30.15 seconds


In [18]:
start_time = time.time()

joined_df_broadcast = large_df.join(sf.broadcast(small_df), "product_id")
joined_df_broadcast.show(5)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

+----------+-------------+-----------+
|product_id|product_names|   category|
+----------+-------------+-----------+
|         1|    Product_1|Electronics|
|         2|    Product_2|   Clothing|
|         3|    Product_3|       Toys|
|         4|    Product_4|  Groceries|
|         5|    Product_5|      Books|
+----------+-------------+-----------+

Time taken: 28.35 seconds


In [20]:
print("Without broadcasting")
print(large_df.join(small_df, "product_id").explain())

print("With broadcasting")
print(large_df.join(sf.broadcast(small_df), "product_id").explain())

Without broadcasting
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [product_id#95L, product_names#96, category#109]
   +- SortMergeJoin [product_id#95L], [product_id#108L], Inner
      :- Sort [product_id#95L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(product_id#95L, 200), ENSURE_REQUIREMENTS, [plan_id=536]
      :     +- Filter isnotnull(product_id#95L)
      :        +- Scan ExistingRDD[product_id#95L,product_names#96]
      +- Sort [product_id#108L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(product_id#108L, 200), ENSURE_REQUIREMENTS, [plan_id=537]
            +- Filter isnotnull(product_id#108L)
               +- Scan ExistingRDD[product_id#108L,category#109]


None
With broadcasting
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [product_id#95L, product_names#96, category#109]
   +- BroadcastHashJoin [product_id#95L], [product_id#108L], Inner, BuildRight, false
      :- Filter isnotnull(product_id#

#### week_2_pyspark_homework.ipynb

In [27]:
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf


spark = (
    SparkSession
    .builder
    .appName("DE - Week 1. Homework")
    .master("local[*]")
    .getOrCreate()
)
